In [3]:
!pip install tensorflow_datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.3 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21548 sha256=84e090ee5c9cf37458e476737c00f16c3ca758d605ece963a8f101550992438c
  Stored in directory: /home/jupyter/.cache/pip/wheels/54/4e/28/3ed0e1c8a752867445bab994d2340724928aa3ab059c57c8db
Successfully built promise

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install tensorflow_recommenders


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
pip uninstall scann -y

Found existing installation: scann 1.4.0
Uninstalling scann-1.4.0:
  Successfully uninstalled scann-1.4.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -q tensorflow==2.15.0


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -q numpy==2.2.4

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 8.0.0 requires numpy<2,>=1, but you have numpy 2.2.4 which is incompatible.
numba 0.61.0 requires numpy<2.2,>=1.24, but you have numpy 2.2.4 which is incompatible.
pyarrow 15.0.2 requires numpy<2,>=1.16.6, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 2.2.4 which is incompatible.
ydata-profiling 4.12.2 requires numpy<2.2,>=1.16.0, but you have numpy 2.2.4 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pprint
import tempfile
from typing import Dict,Text
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

2025-03-30 02:06:33.971857: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-30 02:06:34.030350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-30 02:06:34.030393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-30 02:06:34.032333: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 02:06:34.043641: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-30 02:06:34.045252: I tensorflow/core/platform/cpu_feature_guard.cc:1

Prepare Dataset

In [2]:
#Rating data
ratings = tfds.load("movielens/100k-ratings",split="train")
#Features of all candidate data
movies = tfds.load("movielens/100k-movies",split="train")

In [3]:
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


2025-03-30 02:06:44.137925: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [4]:
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


2025-03-30 02:06:46.598379: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [5]:
ratings = ratings.map(lambda x:{
    "movie_title" : x["movie_title"],
    "user_id" : x["user_id"],
})

movies = movies.map(lambda x:x["movie_title"])

In [6]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000,seed=42,reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [7]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_00_000).map(lambda x:x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [8]:
unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

In [9]:
unique_user_ids[:10]

array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
       b'106', b'107'], dtype=object)

Implementing Model

In [10]:
embedding_dimension = 32

In [11]:
user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids,mask_token=None),
    tf.keras.layers.Embedding(len(unique_user_ids)+1,embedding_dimension)
])

In [12]:
movie_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles,mask_token=None),
    tf.keras.layers.Embedding(len(unique_movie_titles)+1,embedding_dimension)
])

In [13]:
metrics = tfrs.metrics.FactorizedTopK(
    candidates = movies.batch(128).map(movie_model)
)

In [14]:
task = tfrs.tasks.Retrieval(metrics=metrics)

The Full Model

In [15]:
class MovielensModel(tfrs.Model):
    
    def __init__(self,user_model,movie_model):
        super().__init__()
        self.movie_model : tf.keras.Model = movie_model
        self.user_model : tf.keras.Model = user_model
        self.task : tf.keras.layers.Layer = task
        
    def compute_loss(self,features:Dict[Text,tf.Tensor],training=False) ->tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])
        
        return self.task(user_embeddings,movie_embeddings)

Fitting and Evaluating

In [16]:
model = MovielensModel(user_model,movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [17]:
cache_trained = train.shuffle(100_000).batch(8192).cache()
cache_test = test.batch(4096).cache()

In [21]:
model.fit(cache_trained,epochs=3)

Epoch 1/3
10/10 [==============================] - 18s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0013 - factorized_top_k/top_5_categorical_accuracy: 0.0099 - factorized_top_k/top_10_categorical_accuracy: 0.0216 - factorized_top_k/top_50_categorical_accuracy: 0.1001 - factorized_top_k/top_100_categorical_accuracy: 0.1782 - loss: 69811.3146 - regularization_loss: 0.0000e+00 - total_loss: 69811.3146
Epoch 2/3
10/10 [==============================] - 13s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0031 - factorized_top_k/top_5_categorical_accuracy: 0.0195 - factorized_top_k/top_10_categorical_accuracy: 0.0397 - factorized_top_k/top_50_categorical_accuracy: 0.1686 - factorized_top_k/top_100_categorical_accuracy: 0.2899 - loss: 67488.4964 - regularization_loss: 0.0000e+00 - total_loss: 67488.4964
Epoch 3/3
10/10 [==============================] - 16s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0035 - factorized_top_k/top_5_categorical_accuracy: 0.0237

In [22]:
model.evaluate(cache_test,return_dict=True)

5/5 [==============================] - 6s 515ms/step - factorized_top_k/top_1_categorical_accuracy: 7.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0091 - factorized_top_k/top_10_categorical_accuracy: 0.0209 - factorized_top_k/top_50_categorical_accuracy: 0.1211 - factorized_top_k/top_100_categorical_accuracy: 0.2360 - loss: 31076.4505 - regularization_loss: 0.0000e+00 - total_loss: 31076.4505


{'factorized_top_k/top_1_categorical_accuracy': 0.000750000006519258,
 'factorized_top_k/top_5_categorical_accuracy': 0.009100000374019146,
 'factorized_top_k/top_10_categorical_accuracy': 0.020899999886751175,
 'factorized_top_k/top_50_categorical_accuracy': 0.12110000103712082,
 'factorized_top_k/top_100_categorical_accuracy': 0.23604999482631683,
 'loss': 28237.96484375,
 'regularization_loss': 0,
 'total_loss': 28237.96484375}

Model Prediction

In [24]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

index.index_from_dataset(
    tf.data.Dataset.zip(movies.batch(100),movies.batch(100).map(model.movie_model)))

_,titles = index(tf.constant(["42"]))

print(f"Recommendations for user 42: {titles[0,:3]}")

Recommendations for user 42: [b'Homeward Bound: The Incredible Journey (1993)'
 b'Grumpier Old Men (1995)' b'101 Dalmatians (1996)']


In [25]:
ot = model.user_model.predict(tf.constant(["42"]))
ot

1/1 [==============================] - 0s 71ms/step


array([[-0.22899199, -0.16064022,  0.07679612,  0.409632  , -0.26854253,
         0.38478675, -0.26453745,  0.03933088,  0.07499922,  0.24885684,
         0.45341435, -0.26657316,  0.46365067,  0.22015373,  0.30962116,
         0.0554018 ,  0.05350643, -0.11967805,  0.5630117 , -0.5616781 ,
        -0.32834166,  0.21279982, -0.01844121, -0.3354391 , -0.01276881,
         0.20476872,  0.32642972,  0.3223742 ,  0.31260163, -0.25470287,
         0.5752901 , -0.42915764]], dtype=float32)

In [26]:
otM = model.movie_model.predict(tf.constant(["Harry Potter"]))
otM

1/1 [==============================] - 0s 47ms/step


array([[-4.1875530e-02, -3.3552758e-03, -1.1418484e-02, -4.1328799e-02,
         3.7990510e-05, -6.1395392e-03, -6.6739805e-03,  4.1362967e-02,
        -8.4338896e-03, -4.3423273e-02,  3.2257233e-02,  4.4082556e-02,
        -2.8233742e-02,  2.6133407e-02, -1.0690115e-02, -4.7576822e-02,
        -3.4383632e-02,  1.4381114e-02,  2.5310367e-04, -2.8390264e-02,
         4.5575093e-02, -2.3143947e-02,  2.3847070e-02, -3.9604805e-02,
        -2.8399551e-02,  3.8856816e-02, -8.2620867e-03,  1.4367703e-02,
         1.6659919e-02,  1.9457843e-02,  4.6380647e-03,  6.5726154e-03]],
      dtype=float32)

In [22]:
# User Model
class UserModel(tf.keras.Model):
    def __init__(self,unique_user_ids):
        super().__init__()
        self.string_lookup = tf.keras.layers.StringLookup(vocabulary= unique_user_ids, mask_token=None)
        self.embedding = tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        self.dense = tf.keras.layers.Dense(embedding_dimension, activation="relu")
    
    def call(self, inputs):
        user_vocab = self.string_lookup(inputs)
        user_embedding = self.embedding(user_vocab)
        return self.dense(user_embedding)


In [23]:
#Movie Model
class MovieModel(tf.keras.Model):
    def __init__(self,unique_movie_titles):
        super().__init__()
        self.string_lookup = tf.keras.layers.StringLookup(vocabulary=unique_movie_titles, mask_token=None)
        self.embedding = tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
        self.dense = tf.keras.layers.Dense(embedding_dimension, activation="relu")

    def call(self, inputs):
        movie_id = inputs  # Preserve original movie ID
        movie_index = self.string_lookup(inputs)
        embedding = self.embedding(movie_index)
        embedding = self.dense(embedding)
        return {"movie_id": movie_id, "embedding": embedding}

In [25]:
movie_model2 = MovieModel(unique_movie_titles)

In [26]:
metrics2 = tfrs.metrics.FactorizedTopK(
    candidates = movies.batch(128).map(lambda x: movie_model2(x)["embedding"])
)

In [27]:
task2 = tfrs.tasks.Retrieval(metrics=metrics2)

In [34]:
# Full Two-Tower Model
class TwoTowerModel(tfrs.models.Model):
    def __init__(self, user_modell, movie_modell, task):
        super().__init__()
        self.user_model = user_modell
        self.movie_model = movie_modell
        self.task = task2

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        movie_output = self.movie_model(features["movie_title"])
        movie_embeddings = movie_output["embedding"]
        return self.task(user_embeddings, movie_embeddings)

In [35]:
user_model2 = UserModel(unique_user_ids)

In [36]:
# Instantiate and compile model
model = TwoTowerModel(user_model2,movie_model2,task2)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))



In [37]:
# Train model
model.fit(cache_trained, epochs=3)



Epoch 1/3
10/10 [==============================] - 19s 1s/step - factorized_top_k/top_1_categorical_accuracy: 3.7500e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0026 - factorized_top_k/top_10_categorical_accuracy: 0.0054 - factorized_top_k/top_50_categorical_accuracy: 0.0298 - factorized_top_k/top_100_categorical_accuracy: 0.0609 - loss: 70367.4872 - regularization_loss: 0.0000e+00 - total_loss: 70367.4872
Epoch 2/3
10/10 [==============================] - 15s 2s/step - factorized_top_k/top_1_categorical_accuracy: 5.7500e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0037 - factorized_top_k/top_10_categorical_accuracy: 0.0081 - factorized_top_k/top_50_categorical_accuracy: 0.0402 - factorized_top_k/top_100_categorical_accuracy: 0.0795 - loss: 70365.2855 - regularization_loss: 0.0000e+00 - total_loss: 70365.2855
Epoch 3/3
10/10 [==============================] - 13s 1s/step - factorized_top_k/top_1_categorical_accuracy: 7.7500e-04 - factorized_top_k/top_5_categorical_accu

In [38]:
# Save User and Movie Models for Deployment
user_model_trained = model.user_model
movie_model_trained = model.movie_model



In [39]:
# Save models separately for Vertex AI
user_model_trained.save("user_model")
movie_model_trained.save("movie_model")

INFO:tensorflow:Assets written to: user_model/assets


INFO:tensorflow:Assets written to: user_model/assets


INFO:tensorflow:Assets written to: movie_model/assets


INFO:tensorflow:Assets written to: movie_model/assets


In [41]:
loaded_movie_model = tf.keras.models.load_model("movie_model")

In [42]:
random_movie = "The Woman in the back yard"
prediction = loaded_movie_model(tf.constant([random_movie]))
print(prediction)

{'embedding': <tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.0315778 , 0.04169   , 0.03026812, 0.03663844, 0.        ,
        0.        , 0.00309834, 0.        , 0.01251587, 0.        ,
        0.08749538, 0.02146452, 0.02986801, 0.03569703, 0.04205424,
        0.        , 0.0388275 , 0.        , 0.        , 0.01729238,
        0.0341426 , 0.0080914 , 0.01027098, 0.00677884, 0.05901616,
        0.        , 0.01843048, 0.04870323, 0.07441176, 0.02373037,
        0.03263981, 0.        ]], dtype=float32)>, 'movie_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'The Woman in the back yard'], dtype=object)>}


In [43]:
loaded_query_model = tf.keras.models.load_model("user_model")

In [46]:
prediction2 = loaded_query_model(tf.constant(["42"]))
print(prediction2)

tf.Tensor(
[[0.         0.         0.00719832 0.         0.05396531 0.
  0.02203207 0.00657738 0.0474218  0.07067144 0.01534967 0.04755773
  0.         0.04117741 0.04987399 0.00858011 0.00704047 0.
  0.05137636 0.         0.00088939 0.         0.00930218 0.
  0.01854408 0.00101339 0.         0.08081911 0.07031809 0.02777579
  0.         0.04631174]], shape=(1, 32), dtype=float32)
